# Advanced Operations for LLM Input / Output with LangChain

This notebook covers the following operations:

- LLM Cost Monitoring
- Caching
- Streaming

## Install OpenAI, HuggingFace and LangChain dependencies

In [1]:
!pip install langchain==0.3.11
!pip install langchain-openai==0.2.12
!pip install langchain-community==0.3.11

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.9/326.9 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.3.11
    Uninstalling langsmith-0.3.11:
      Successfully uninstalled langsmith-0.3.11
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.19
    Uninstalling langchain-0.3.19:
      Successfully uninstalled langchain-0.3.19
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00


## Setup necessary system environment variables

In [2]:
from google.colab import userdata
import os

os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HF_TOKEN')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

## Chat Models and LLMs

Large Language Models (LLMs) are a core component of LangChain. LangChain does not implement or build its own LLMs. It provides a standard API for interacting with almost every LLM out there.

There are lots of LLM providers (OpenAI, Hugging Face, etc) - the LLM class is designed to provide a standard interface for all of them.

## Accessing Commercial LLMs like ChatGPT

In [3]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

## Tracking LLM Costs

Typically LLMs like ChatGPT charge you based on the number of tokens per request and response. You can track your token usage for specific calls. It is currently only implemented for the OpenAI API.

In [4]:
from langchain_community.callbacks import get_openai_callback

prompt = """Explain Generative AI in one line"""

with get_openai_callback() as cb:
    response = chatgpt.invoke(prompt)
    print(response.content)
    print(cb)

Generative AI refers to algorithms that can create new content, such as text, images, or music, by learning patterns from existing data.
Tokens Used: 42
	Prompt Tokens: 14
	Completion Tokens: 28
Successful Requests: 1
Total Cost (USD): $1.89e-05


In [5]:
cb.total_tokens

42

In [6]:
cb.prompt_tokens, cb.completion_tokens

(14, 28)

In [7]:
cb.total_cost

1.89e-05

## Caching in LangChain

LangChain includes an optional caching layer for language model APIs (LLMs). This caching feature is beneficial for two main reasons:

1. **Cost Efficiency:** By caching responses, you reduce the number of API calls made to LLM providers, especially helpful if you are frequently requesting the same completions. This can significantly lower costs.

2. **Performance Improvement:** Caching can enhance your application's speed by decreasing the need for repeated API calls to the LLM provider, making interactions quicker and more efficient.


#### InMemoryCache

In [8]:
%%time
# integrations with other caching tools:
# https://api.python.langchain.com/en/latest/community_api_reference.html#module-langchain_community.cache
from langchain.cache import InMemoryCache
from langchain.globals import set_llm_cache

set_llm_cache(InMemoryCache())

# The first time, it is not yet in cache, so it should take longer

from langchain_core.prompts import ChatPromptTemplate

prompt = """Explain to me what is mortgage"""

chat_template = ChatPromptTemplate.from_template(prompt)

chatgpt.invoke(chat_template.format())

CPU times: user 1.24 s, sys: 73.7 ms, total: 1.32 s
Wall time: 10.1 s


AIMessage(content="A mortgage is a type of loan specifically used to purchase real estate, such as a home or property. In a mortgage agreement, the borrower (the person buying the property) receives funds from a lender (usually a bank or financial institution) to buy the property. In return, the borrower agrees to repay the loan amount, plus interest, over a specified period, typically 15 to 30 years.\n\nKey features of a mortgage include:\n\n1. **Collateral**: The property itself serves as collateral for the loan. If the borrower fails to make the required payments, the lender has the right to take possession of the property through a legal process called foreclosure.\n\n2. **Down Payment**: Most mortgages require a down payment, which is a percentage of the property's purchase price that the borrower pays upfront. The remaining amount is financed through the mortgage.\n\n3. **Interest Rate**: Mortgages come with either fixed or variable interest rates. A fixed-rate mortgage has a con

In [9]:
%%time
# The second time it is, so it goes faster
chatgpt.invoke(chat_template.format())

CPU times: user 1.24 ms, sys: 0 ns, total: 1.24 ms
Wall time: 1.22 ms


AIMessage(content="A mortgage is a type of loan specifically used to purchase real estate, such as a home or property. In a mortgage agreement, the borrower (the person buying the property) receives funds from a lender (usually a bank or financial institution) to buy the property. In return, the borrower agrees to repay the loan amount, plus interest, over a specified period, typically 15 to 30 years.\n\nKey features of a mortgage include:\n\n1. **Collateral**: The property itself serves as collateral for the loan. If the borrower fails to make the required payments, the lender has the right to take possession of the property through a legal process called foreclosure.\n\n2. **Down Payment**: Most mortgages require a down payment, which is a percentage of the property's purchase price that the borrower pays upfront. The remaining amount is financed through the mortgage.\n\n3. **Interest Rate**: Mortgages come with either fixed or variable interest rates. A fixed-rate mortgage has a con

#### SQLite Cache

In [10]:
# just to remove cache if it already exists, ignore if you get an error message below, that is normal (when cache doesnt exist)
!rm langchain.db

rm: cannot remove 'langchain.db': No such file or directory


In [11]:
# We can do the same thing with a SQLite cache
from langchain.cache import SQLiteCache

set_llm_cache(SQLiteCache(database_path="langchain.db"))

In [12]:
%%time

# The first time, it is not yet in cache, so it should take longer
prompt = """Explain to me what is fractional real estate"""

chat_template = ChatPromptTemplate.from_template(prompt)

chatgpt.invoke(chat_template.format())

CPU times: user 57.5 ms, sys: 6.92 ms, total: 64.4 ms
Wall time: 5.53 s


AIMessage(content='Fractional real estate refers to a model of property ownership where multiple individuals or entities share ownership of a single property. This arrangement allows investors to purchase a fraction or percentage of a property rather than the whole asset, making real estate investment more accessible and affordable.\n\nHere are some key features of fractional real estate:\n\n1. **Shared Ownership**: Multiple owners hold a share of the property, which can range from a small percentage to a larger portion, depending on the investment structure.\n\n2. **Cost-Effective**: By sharing the cost of the property, investors can participate in real estate markets that might otherwise be out of reach due to high prices.\n\n3. **Usage Rights**: Owners typically have the right to use the property for a certain number of days or weeks each year, depending on their ownership share. This is common in vacation properties or timeshares.\n\n4. **Management**: Fractional ownership often in

In [13]:
%%time
# The second time it is, so it goes faster
chatgpt.invoke(chat_template.format())

CPU times: user 2.78 ms, sys: 970 µs, total: 3.75 ms
Wall time: 3.36 ms


AIMessage(content='Fractional real estate refers to a model of property ownership where multiple individuals or entities share ownership of a single property. This arrangement allows investors to purchase a fraction or percentage of a property rather than the whole asset, making real estate investment more accessible and affordable.\n\nHere are some key features of fractional real estate:\n\n1. **Shared Ownership**: Multiple owners hold a share of the property, which can range from a small percentage to a larger portion, depending on the investment structure.\n\n2. **Cost-Effective**: By sharing the cost of the property, investors can participate in real estate markets that might otherwise be out of reach due to high prices.\n\n3. **Usage Rights**: Owners typically have the right to use the property for a certain number of days or weeks each year, depending on their ownership share. This is common in vacation properties or timeshares.\n\n4. **Management**: Fractional ownership often in

## Streaming in LLMs

All language model interfaces (LLMs) in LangChain implement the `Runnable` interface, which provides default methods such as `ainvoke`, `batch`, `abatch`, `stream`, and `astream`. This setup equips all LLMs with basic streaming capabilities.

### Streaming Defaults:

- **Synchronous Streaming:** By default, streaming operations return an `Iterator` that yields a single value, the final result from the LLM provider.
- **Asynchronous Streaming:** Similarly, async streaming defaults to returning an `AsyncIterator` with the final result.

### Limitations:

- These default implementations do not support token-by-token streaming. For such detailed streaming, the LLM provider must offer native support. However, the default setup ensures that your code expecting an iterator of tokens will function correctly within these constraints.


In [14]:
prompt = """Explain to me what is mortgage in detail with pros and cons"""
chat_template = ChatPromptTemplate.from_template(prompt)

for chunk in chatgpt.stream(chat_template.format()):
    print(chunk.content)


A
 mortgage
 is
 a
 type
 of
 loan
 specifically
 used
 to
 purchase
 real
 estate
,
 where
 the
 property
 itself
 serves
 as
 collateral
 for
 the
 loan
.
 When
 a
 borrower
 takes
 out
 a
 mortgage
,
 they
 agree
 to
 repay
 the
 loan
 amount
,
 plus
 interest
,
 over
 a
 specified
 period
,
 typically
 
15
 to
 
30
 years
.
 Mort
gages
 are
 commonly
 used
 by
 individuals
 and
 families
 to
 buy
 homes
,
 but
 they
 can
 also
 be
 used
 for
 investment
 properties
 or
 commercial
 real
 estate
.


###
 Key
 Components
 of
 a
 Mortgage



1
.
 **
Principal
**
:
 The
 original
 loan
 amount
 borrowed
 from
 the
 lender
.

2
.
 **
Interest
 Rate
**
:
 The
 cost
 of
 borrowing
 the
 principal
,
 expressed
 as
 a
 percentage
.
 This
 can
 be
 fixed
 (
rem
ains
 the
 same
 throughout
 the
 loan
 term
)
 or
 variable
 (
can
 change
 at
 specified
 times
).

3
.
 **
Term
**
:
 The
 length
 of
 time
 over
 which
 the
 loan
 must
 be
 rep
aid
,
 commonly
 
15
,
 
20
,
 or
 
30
 years
.

4


In [15]:
prompt = """Explain to me what is mortgage in detail with pros and cons"""
chat_template = ChatPromptTemplate.from_template(prompt)

response = []
for chunk in chatgpt.stream(chat_template.format()):
    print(chunk.content, end="")
    response.append(chunk.content)

A mortgage is a type of loan specifically used to purchase real estate, where the property itself serves as collateral for the loan. When a borrower takes out a mortgage, they agree to repay the loan amount, plus interest, over a specified period, typically 15 to 30 years. Mortgages are commonly used by individuals and families to buy homes, but they can also be used for investment properties or commercial real estate.

### Key Components of a Mortgage

1. **Principal**: The original loan amount borrowed from the lender.
2. **Interest**: The cost of borrowing the principal, expressed as a percentage. This can be fixed (stays the same throughout the loan term) or variable (can change at specified times).
3. **Term**: The length of time over which the loan must be repaid, commonly 15, 20, or 30 years.
4. **Down Payment**: The initial amount paid upfront by the borrower, usually expressed as a percentage of the property’s purchase price. A larger down payment can reduce the loan amount an

In [16]:
response[:10]

['',
 'A',
 ' mortgage',
 ' is',
 ' a',
 ' type',
 ' of',
 ' loan',
 ' specifically',
 ' used']

In [17]:
print(''.join(response))

A mortgage is a type of loan specifically used to purchase real estate, where the property itself serves as collateral for the loan. When a borrower takes out a mortgage, they agree to repay the loan amount, plus interest, over a specified period, typically 15 to 30 years. Mortgages are commonly used by individuals and families to buy homes, but they can also be used for investment properties or commercial real estate.

### Key Components of a Mortgage

1. **Principal**: The original loan amount borrowed from the lender.
2. **Interest**: The cost of borrowing the principal, expressed as a percentage. This can be fixed (stays the same throughout the loan term) or variable (can change at specified times).
3. **Term**: The length of time over which the loan must be repaid, commonly 15, 20, or 30 years.
4. **Down Payment**: The initial amount paid upfront by the borrower, usually expressed as a percentage of the property’s purchase price. A larger down payment can reduce the loan amount an